In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2023-07-26 10:12:48--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   243MB/s    in 0.3s    

2023-07-26 10:12:49 (243 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [ ]:
!conda create --name=pyoccenv python=3.9

Solving environment: / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 23.5.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local/envs/pyoccenv

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-5.1          |            1_gnu          21 KB
    ca-certificates-2023.05.30 |       h06a4308_0         120 KB
    ld_impl_linux-64-2.38      |       h1181459_1         654 KB
    libffi-3.4.4               |       h6a678d5_0         142 KB
    libgcc-ng-11.2.0           |       h1234567_1         5.3 MB
    libgomp-11.2.0             |       h1234567_1         474 KB
    libstdcxx-ng-11.2.0        |       h1234567_1         4.7 MB
    ncurses-6.4                |       h6a678d5_0         914 KB
    opens

In [ ]:
!source activate pyoccenv

In [ ]:
!conda install -c conda-forge pythonocc-core=7.7.0

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [ ]:
from OCC.Extend.DataExchange import read_step_file
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool


ModuleNotFoundError: ignored

In [ ]:
filename = "Unnamed1-BodyPad.step"
shapes = read_step_file(filename)

faces = []
edges = []

face_iterator = TopExp_Explorer(shapes, TopAbs_FACE)
while face_iterator.More():
    face = topods.Face(face_iterator.Current())
    faces.append(face)
    face_iterator.Next()

    edge_iterator = TopExp_Explorer(face, TopAbs_EDGE)
    while edge_iterator.More():
        edge = topods.Edge(edge_iterator.Current())
        edges.append(edge)
        edge_iterator.Next()


In [ ]:
vertex_coordinates = []

for edge in edges:
    vertex_iterator = TopExp_Explorer(edge, TopAbs_VERTEX)
    while vertex_iterator.More():
        vertex = topods.Vertex(vertex_iterator.Current())
        point = BRep_Tool.Pnt(vertex)
        vertex_coordinates.append((point.X(), point.Y(), point.Z()))
        vertex_iterator.Next()

print(vertex_coordinates)


In [ ]:
set(vertex_coordinates)

In [ ]:
min_x = min(vertex[0] for vertex in vertex_coordinates)
min_y = min(vertex[1] for vertex in vertex_coordinates)
min_z = min(vertex[2] for vertex in vertex_coordinates)

In [ ]:
normalized_vertex_coordinates = [(vertex[0] - min_x, vertex[1] - min_y, vertex[2] - min_z) for vertex in vertex_coordinates]

In [ ]:
unique_xy_set = {(x, y) for x, y, z in normalized_vertex_coordinates}
print(unique_xy_set)

In [ ]:
!pip install scipy
pip install alphashape

In [ ]:
import json
from scipy.spatial import ConvexHull

def generate_geometry_data(coordinates):
    # Step 1: Generate unique IDs for each point
    points = []
    for i, (x, y) in enumerate(coordinates):
        point = {'id': f'P{i+1}', 'x': x, 'y': y}
        points.append(point)

    # Step 2: Calculate Convex Hull to obtain ordered vertices
        hull = ConvexHull(coordinates)
        ordered_indices = hull.vertices.tolist()
        ordered_indices.append(ordered_indices[0])  # Add first index to end to complete loop
        ordered_vertices = [coordinates[i] for i in ordered_indices]
    # Step 3: Generate list of edges connecting ordered vertices
    edges = []
    for i in range(len(ordered_indices)-1):
        p1 = points[ordered_indices[i]]
        p2 = points[ordered_indices[i+1]]
        edge = {'p1': p1['id'], 'p2': p2['id']}
        edges.append(edge)

    # Step 4: Generate list of line segments formed by edges
    line_segments = []
    for i in range(len(ordered_indices)-1):
        p1 = points[ordered_indices[i]]
        p2 = points[ordered_indices[i+1]]
        line_segment = {'id': f'S{i+1}', 'p1': p1['id'], 'p2': p2['id']}
        line_segments.append(line_segment)

    # Step 5: Generate JSON-formatted output
        data = {
          'profile': {
              # ... (rest of the code remains the same)
              'geometry': {
                'rotation': {'x': 0, 'y': 0, 'z': 0},
                'version': '1.0',
                'dimensions': {
                    'start': {'x': 0, 'y': 0},
                    'end': {'x': ordered_vertices[-1][0], 'y': ordered_vertices[-1][1]}
                },
                'units': {'distance': 'mm', 'mass': 'kg', 'area': 'm2'},
                'facing': 'top',
                'points': points,
                'shape': line_segments,
                'cuts': [],
                'bends': [],
                'radii': [],
                'stamps': [],
                'stampingTools': []
            }
        }
    }

    return json.dumps(data, indent=2)

# Example usage
coordinates = list(unique_xy_set)
geometry_data = generate_geometry_data(coordinates)
print(geometry_data)